# Problema di permutation flowshop

## Job scheduling con algoritmo genetico

Ci sono m macchine e n posti di lavoro. Ogni job contiene esattamente m operazioni. L'i-esima operazione del lavoro deve essere eseguita sulla i-esima macchina. Nessuna macchina può eseguire più di un'operazione contemporaneamente. Per ogni operazione di ogni lavoro, viene specificato il tempo di esecuzione.
Le operazioni all'interno di un lavoro devono essere eseguite nell'ordine specificato. La prima operazione viene eseguita sulla prima macchina, poi (quando la prima operazione è terminata) la seconda operazione sulla seconda macchina, e così via fino all'ennesima operazione. Tuttavia, i lavori possono essere eseguiti in qualsiasi ordine.
L'obiettivo è quello di completare tutti i job nel minore tempo possibile, ovvero minimizzare il makespan.

### Ipotesi iniziali
Si considera un problema dove ci sono m macchine indipendenti, scheduling non preemptive e tempo di rilascio nullo per tutti i job, quindi i job posso essere eseguiti in qualsiasi istante e senza interruzioni.
Il modello di scheduling può essere modellato nel modo seguente:
F_m || C_max

In [1]:
from functions import *
import numpy as np
import re
import time
import csv

population_number: int = 60
iterations_number: int = 35 # numero di generazioni
p_mutation: float = 0.05    # probabilità di mutazione

bench_data = []             # matrice tempi di esecuzione job-macchina

In [2]:
dictionary = {}
filename = "taillard_20x10.txt"
file = open(filename, "rt")
with file as f:
    for line in f:
       (key, val) = line.split(':')
       dictionary[key] = val    
#print(dictionary)
file.close()

In [3]:
n_jobs = int(dictionary['number of jobs'])
n_machines = int(dictionary['number of machines'])
best_upper_bound = int(dictionary['upper bound'])
#print(n_jobs)
#print(n_machines)
#print(best_upper_bound)

In [4]:

processing_times = re.findall('\[(.*?)\]', dictionary['processing times'])
#print(processing_times)

In [5]:
i = 1
for machine_times in processing_times:
    list_times = machine_times.split(' ')
    list_times = [int(x) for x in list_times]
    bench_data.append(list_times)
    print('machine' + str(i) + ': ')
    print(list_times)
    i = i + 1
#print('\n')
#print('bench_data: ')
#print(bench_data)

machine1: 
[74, 21, 58, 4, 21, 28, 58, 83, 31, 61, 94, 44, 97, 94, 66, 6, 37, 22, 99, 83]
machine2: 
[28, 3, 27, 61, 34, 76, 64, 87, 54, 98, 76, 41, 70, 43, 42, 79, 88, 15, 49, 72]
machine3: 
[89, 52, 56, 13, 7, 32, 32, 98, 46, 60, 23, 87, 7, 36, 26, 85, 7, 34, 36, 48]
machine4: 
[60, 88, 26, 58, 76, 98, 29, 47, 79, 26, 19, 48, 95, 78, 77, 90, 24, 10, 85, 55]
machine5: 
[54, 66, 12, 57, 70, 82, 99, 84, 16, 41, 23, 11, 68, 58, 30, 5, 5, 39, 58, 31]
machine6: 
[92, 11, 54, 97, 57, 53, 65, 77, 51, 36, 53, 19, 54, 86, 40, 56, 79, 74, 24, 3]
machine7: 
[9, 8, 88, 72, 27, 22, 50, 2, 49, 82, 93, 96, 43, 13, 60, 11, 37, 91, 84, 67]
machine8: 
[4, 18, 25, 28, 95, 51, 84, 18, 6, 90, 69, 61, 57, 5, 75, 4, 38, 28, 4, 80]
machine9: 
[25, 15, 91, 49, 56, 10, 62, 70, 76, 99, 58, 83, 84, 64, 74, 14, 18, 48, 96, 86]
machine10: 
[15, 84, 8, 30, 95, 79, 9, 91, 76, 26, 42, 66, 70, 91, 67, 3, 98, 4, 71, 62]


In [6]:
processing_time = [] # tempi di esecuzione del job j sulla macchina i
for i in range(n_jobs):
    temp = []
    for j in range(n_machines):
        temp.append(bench_data[j][i])
    print('job' + str(i + 1) + ': ')
    print(temp)
    processing_time.append(temp)


#print(processing_time)

job1: 
[74, 28, 89, 60, 54, 92, 9, 4, 25, 15]
job2: 
[21, 3, 52, 88, 66, 11, 8, 18, 15, 84]
job3: 
[58, 27, 56, 26, 12, 54, 88, 25, 91, 8]
job4: 
[4, 61, 13, 58, 57, 97, 72, 28, 49, 30]
job5: 
[21, 34, 7, 76, 70, 57, 27, 95, 56, 95]
job6: 
[28, 76, 32, 98, 82, 53, 22, 51, 10, 79]
job7: 
[58, 64, 32, 29, 99, 65, 50, 84, 62, 9]
job8: 
[83, 87, 98, 47, 84, 77, 2, 18, 70, 91]
job9: 
[31, 54, 46, 79, 16, 51, 49, 6, 76, 76]
job10: 
[61, 98, 60, 26, 41, 36, 82, 90, 99, 26]
job11: 
[94, 76, 23, 19, 23, 53, 93, 69, 58, 42]
job12: 
[44, 41, 87, 48, 11, 19, 96, 61, 83, 66]
job13: 
[97, 70, 7, 95, 68, 54, 43, 57, 84, 70]
job14: 
[94, 43, 36, 78, 58, 86, 13, 5, 64, 91]
job15: 
[66, 42, 26, 77, 30, 40, 60, 75, 74, 67]
job16: 
[6, 79, 85, 90, 5, 56, 11, 4, 14, 3]
job17: 
[37, 88, 7, 24, 5, 79, 37, 38, 18, 98]
job18: 
[22, 15, 34, 10, 39, 74, 91, 28, 48, 4]
job19: 
[99, 49, 36, 85, 58, 24, 84, 4, 96, 71]
job20: 
[83, 72, 48, 55, 31, 3, 67, 80, 86, 62]


In [7]:
start_time = time.time()
# creazione popolazione
population = initialize_population(population_number, n_jobs)
#print(population)

for evaluation in range(iterations_number):
    print('---- creazione ' + str(evaluation) + ' generazione ----\n')
    parents = get_parents(population, processing_time, n_jobs, n_machines)
    childs = []
    # operatore di crossover per generare nuove sequenze di scheduling
    childs.append(crossover(parents))
   # print('\n')
   # print(childs)

    # operatore di mutazione
    mutated_childs = []
    for child in childs:
        r = np.random.rand()
        if r < p_mutation:
            mutated_child = mutation(child)
            mutated_childs.append(mutated_child)

    childs.extend(mutated_childs)
    if len(childs) > 0:
        update_population(population, childs, processing_time, n_jobs, n_machines)
    print('---- generazione ' + str(evaluation) + ' creata ----\n')


---- creazione 0 generazione ----

fitness cumulata per sequenza: 
(0.019049283467319118, [0, 13, 11, 7, 19, 3, 1, 9, 2, 6, 5, 4, 17, 16, 10, 14, 18, 15, 8, 12])
(0.03067109402306886, [13, 5, 14, 12, 16, 19, 1, 15, 6, 17, 0, 7, 9, 18, 2, 4, 3, 8, 11, 10])
(0.03871024117441454, [12, 15, 1, 14, 2, 3, 11, 9, 19, 7, 17, 18, 10, 16, 5, 8, 4, 13, 0, 6])
(0.05330304089479203, [11, 18, 19, 7, 1, 5, 9, 4, 10, 0, 15, 2, 3, 12, 6, 8, 16, 17, 13, 14])
(0.06597343586158685, [19, 18, 3, 8, 1, 14, 5, 4, 6, 13, 12, 0, 16, 11, 2, 7, 10, 17, 9, 15])
(0.0699930094372597, [11, 18, 12, 13, 15, 6, 4, 0, 8, 17, 19, 2, 9, 14, 10, 1, 3, 5, 7, 16])
(0.08502271932890598, [5, 6, 11, 1, 19, 18, 10, 2, 14, 15, 16, 12, 8, 3, 17, 9, 0, 13, 7, 4])
(0.10852848654316673, [14, 5, 17, 7, 3, 18, 15, 9, 6, 10, 16, 1, 2, 0, 4, 19, 13, 11, 12, 8])
(0.11962600489339392, [3, 0, 17, 9, 15, 6, 2, 19, 12, 7, 5, 1, 13, 8, 4, 18, 14, 10, 16, 11])
(0.12862635442153092, [0, 18, 15, 5, 2, 11, 7, 3, 8, 17, 6, 1, 12, 10, 14, 9, 4, 13, 16

---- generazione 28 creata ----

---- creazione 29 generazione ----

fitness cumulata per sequenza: 
(0.008076728924785462, [0, 13, 11, 7, 19, 3, 1, 9, 2, 6, 5, 4, 17, 16, 10, 14, 18, 15, 8, 12])
(0.008413259296651522, [5, 6, 11, 1, 19, 18, 10, 2, 14, 15, 16, 12, 8, 3, 17, 9, 0, 13, 7, 4])
(0.02507151270402154, [14, 5, 17, 7, 3, 18, 15, 9, 6, 10, 16, 1, 2, 0, 4, 19, 13, 11, 12, 8])
(0.04795557799091368, [5, 14, 3, 15, 19, 9, 12, 8, 11, 7, 10, 1, 4, 0, 2, 16, 6, 18, 13, 17])
(0.07656065959952886, [9, 14, 17, 6, 19, 12, 4, 16, 8, 0, 1, 7, 2, 18, 13, 5, 10, 11, 3, 15])
(0.09658421672555949, [11, 4, 17, 9, 19, 13, 14, 16, 15, 5, 0, 1, 18, 10, 8, 6, 12, 7, 3, 2])
(0.09658421672555949, [16, 5, 4, 19, 0, 1, 15, 10, 3, 7, 6, 2, 12, 17, 8, 9, 14, 13, 18, 11])
(0.12838633686690223, [8, 13, 4, 2, 1, 0, 7, 10, 18, 19, 14, 9, 12, 11, 17, 5, 16, 3, 6, 15])
(0.1452128554602053, [16, 2, 9, 19, 1, 14, 12, 18, 11, 4, 10, 17, 7, 3, 8, 15, 0, 13, 6, 5])
(0.16052498738011106, [5, 10, 8, 3, 13, 9, 16, 17, 4

---- generazione 30 creata ----

---- creazione 31 generazione ----

fitness cumulata per sequenza: 
(0.007033796534568536, [0, 13, 11, 7, 19, 3, 1, 9, 2, 6, 5, 4, 17, 16, 10, 14, 18, 15, 8, 12])
(0.02281694973408818, [14, 5, 17, 7, 3, 18, 15, 9, 6, 10, 16, 1, 2, 0, 4, 19, 13, 11, 12, 8])
(0.04494767541602333, [5, 14, 3, 15, 19, 9, 12, 8, 11, 7, 10, 1, 4, 0, 2, 16, 6, 18, 13, 17])
(0.07291130554125923, [9, 14, 17, 6, 19, 12, 4, 16, 8, 0, 1, 7, 2, 18, 13, 5, 10, 11, 3, 15])
(0.092125579001544, [11, 4, 17, 9, 19, 13, 14, 16, 15, 5, 0, 1, 18, 10, 8, 6, 12, 7, 3, 2])
(0.12334877337450678, [8, 13, 4, 2, 1, 0, 7, 10, 18, 19, 14, 9, 12, 11, 17, 5, 16, 3, 6, 15])
(0.13930348258706468, [16, 2, 9, 19, 1, 14, 12, 18, 11, 4, 10, 17, 7, 3, 8, 15, 0, 13, 6, 5])
(0.15371418768227826, [5, 10, 8, 3, 13, 9, 16, 17, 4, 0, 14, 12, 19, 2, 6, 7, 15, 1, 11, 18])
(0.17944758963801682, [0, 14, 11, 19, 12, 4, 5, 10, 16, 13, 6, 7, 2, 8, 18, 9, 1, 3, 17, 15])
(0.20243609538514326, [4, 9, 3, 17, 2, 16, 12, 7, 8, 1

In [8]:
makespans = []
for individual in population:
    ind_makespan = (calc_makespan(individual, processing_time, n_jobs, n_machines), individual)
    makespans.append(ind_makespan)
makespans.sort(key=lambda x: x[0])

#best_seq = list(map(int, makespans[0][1]))
#best_makespan = get_makespan(n_machines, best_seq, bench_data)
best_seq = makespans[0][1]
best_makespan = makespans[0][0]
end_time = time.time()

In [9]:
for ms in makespans:
    print(ms)

(1825, [8, 1, 4, 11, 16, 9, 5, 13, 6, 2, 3, 7, 19, 15, 10, 18, 14, 17, 12, 0])
(1828, [3, 5, 13, 18, 2, 8, 0, 15, 14, 16, 1, 19, 4, 11, 7, 9, 12, 6, 10, 17])
(1862, [8, 1, 18, 17, 7, 16, 14, 9, 11, 2, 5, 10, 3, 19, 4, 12, 13, 15, 6, 0])
(1862, [8, 1, 18, 17, 7, 16, 14, 9, 11, 2, 5, 10, 3, 19, 4, 12, 13, 15, 6, 0])
(1862, [3, 12, 2, 8, 0, 4, 5, 9, 13, 6, 11, 16, 18, 14, 17, 15, 1, 19, 7, 10])
(1862, [8, 1, 18, 17, 7, 16, 14, 9, 11, 2, 5, 10, 3, 19, 4, 12, 13, 15, 6, 0])
(1862, [8, 1, 18, 17, 7, 16, 14, 9, 11, 2, 5, 10, 3, 19, 4, 12, 13, 15, 6, 0])
(1872, [8, 13, 4, 2, 1, 0, 7, 10, 18, 19, 14, 9, 12, 11, 17, 5, 16, 3, 6, 15])
(1891, [9, 14, 17, 6, 19, 12, 4, 16, 8, 0, 1, 7, 2, 18, 13, 5, 10, 11, 3, 15])
(1892, [8, 1, 18, 17, 16, 9, 5, 13, 6, 2, 3, 7, 19, 15, 10, 14, 11, 4, 12, 0])
(1897, [1, 17, 14, 9, 11, 4, 5, 10, 16, 13, 6, 7, 2, 12, 18, 3, 19, 8, 15, 0])
(1904, [0, 14, 11, 19, 12, 4, 5, 10, 16, 13, 6, 7, 2, 8, 18, 9, 1, 3, 17, 15])
(1912, [8, 1, 17, 7, 16, 19, 18, 15, 11, 2, 4, 9, 12

In [10]:
print(best_makespan)

1825


In [11]:
print(best_seq)

[8, 1, 4, 11, 16, 9, 5, 13, 6, 2, 3, 7, 19, 15, 10, 18, 14, 17, 12, 0]


In [12]:
print("--- %s secondi ---" % (end_time - start_time))

--- 0.26293110847473145 secondi ---


In [28]:
dic = [{'instance_dim':filename, 'result': best_makespan, 'execution_time':(end_time - start_time)}]
run_info = ['instance_dim', 'result', 'execution_time']

isfile = os.path.isfile('results.csv')
if not isfile:
print('non esiste il file')
with open('results.csv', 'a') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = run_info)
    writer.writeheader()
    writer.writerows(dic)
else:
with open('results.csv', 'a') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = run_info)
    writer.writerows(dic)